In [31]:
import sys
import os
import pandas as pd
from datetime import datetime, timedelta

# Add the src directory to the system path and import functions
sys.path.append(os.path.abspath('../src'))
from schedule_weather import generate_activity_list, Scheduler
from weather_constraints import get_daylight_windows, get_tide_windows

from plot_weather import plot_weather

In [32]:
#read in activity data from excel
#update to be relative to your project structure
main_input_folder_path = r"C:\Users\webbb\VolkerWessels UK\Grp_VMS GeoSeacore Collaboration Space - SZC\2 Internal WIP\06. MST's\Weather Windows"
path = os.path.join(main_input_folder_path, "WeatherWindows.xlsx")
act_df = pd.read_excel(path, sheet_name="Activities", skiprows=1)
constraints_df = pd.read_excel(path, sheet_name="Constraints", skiprows=1)

activities = generate_activity_list(act_df, constraints_df)

c:\Users\webbb\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning:

Data Validation extension is not supported and will be removed



In [33]:
# 1. Define target activity and completion date
target_activity = "Pilot Punch Out"
input_completion_date = datetime(2025, 12, 14, 6, 0)

In [34]:
# 2. Generate daylight and tidal windows around target date ±7 days
window_data_start = input_completion_date - timedelta(days=5)
window_data_end = input_completion_date + timedelta(days=7)

daylight_windows = get_daylight_windows(
    start_date=window_data_start,
    end_date=window_data_end,
    save_csv=None
)

tidal_data_path = os.path.join(main_input_folder_path, "Reference/harwich_tide.csv")
hw_windows, lw_windows, events = get_tide_windows(
    tidal_data_path,
    start_date=window_data_start,
    end_date=window_data_end,
    skiprows=2,
    datetime_col="Date/Time",
    height_col="Height(m)",
    slack_window_after=1.5,
    slack_window_before=1.5,
    plot=False,
    save_csv=None
)

c:\Users\webbb\Documents\python_local\WeatherOpsPlanner\src\weather_constraints.py:56: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



Loaded data from 2025-10-01 00:00:00 to 2026-10-31 23:50:00 with 57024 records.
Filtered data from 2025-12-09 06:00:00 to 2025-12-21 06:00:00 with 1729 records.


In [35]:
# 3. Create Scheduler instance with activities and environmental windows
scheduler = Scheduler(
    activities=activities,
    daylight_windows=daylight_windows,   # List of (start, end) daylight datetime pairs
    hw_tide_windows=hw_windows,           # High/slack tide windows as datetime pairs
    lw_tide_windows=lw_windows            # Low tide windows as datetime pairs
)

In [36]:
# 4. Run backward scheduling from target completion date for target activity
scheduled_activities = scheduler.schedule_around_target(target_activity, input_completion_date)



In [37]:
# 5. Export scheduled activities to pandas DataFrame for analysis
schedule_df = scheduler.to_dataframe()

# 6. Calculate total schedule duration: from earliest start to latest end of activities
total_duration = schedule_df['End'].max() - schedule_df['Start'].min()

# 7. Print total schedule duration
print(f"Total duration of the schedule: {total_duration}")

# for i, row in schedule_df.iterrows():
#     print(row['Name'])
#     print(row["Constraints"])

# 8. Display the first few scheduled activities in the DataFrame
schedule_df.head(20)


Total duration of the schedule: 5 days 00:37:00


,ID,Name,Start,End,Duration,Group,Description,Predecessor IDs,Successor IDs,Constraints
0,PM01,Marine spread readiness,2025-12-13 10:40:00,2025-12-13 15:40:00,5.00,Pipe Management,Mobilisation of Leask marine spread,,HDD01,"{'daylight_required': True, 'tide_window_requi..."
1,CTV01,Personel transfer prior to pipe pull,2025-12-13 13:40:00,2025-12-13 15:40:00,2.00,Crew Transfer Vessel,CTV transfer of Personnel to JUB,,HDD01,"{'daylight_required': True, 'tide_window_requi..."
2,HDD01,Pilot Punch Out,2025-12-14 06:00:00,2025-12-14 09:00:00,3.00,Horizontal Directional Drilling,Punch out of pilot,"PM01, CTV01","CTV02, HDD02, D01","{'daylight_required': False, 'tide_window_requ..."
3,CTV02,Bathy Survey,2025-12-14 09:00:00,2025-12-14 12:00:00,3.00,Crew Transfer Vessel,Bathymetric survey of pilot to confirm bore lo...,HDD01,JUB01,"{'daylight_required': False, 'tide_window_requ..."
4,HDD02,Pull Back Pilot,2025-12-14 09:00:00,2025-12-14 15:00:00,6.00,Horizontal Directional Drilling,Pull back of pilot,HDD01,HDD03,"{'daylight_required': False, 'tide_window_requ..."
5,PM02,Unmoor Pipe,2025-12-16 09:20:00,2025-12-16 10:20:00,1.00,Pipe Management,Unmooring of pipe from Harwich mooring and hoo...,HDD08,PM03,"{'daylight_required': False, 'tide_window_requ..."
6,JUB01,JUB positioning,2025-12-14 15:30:00,2025-12-15 01:30:00,10.00,Jack-up barge,Postioning of JUB on site (including transitio...,CTV02,JUB02,"{'daylight_required': False, 'tide_window_requ..."
7,JUB02,JUB preparation,2025-12-15 01:30:00,2025-12-15 05:30:00,4.00,Jack-up barge,Readying of JUB to receive PBA,JUB01,W01,"{'daylight_required': False, 'tide_window_requ..."
8,HDD03,Switch to reaming head,2025-12-14 15:00:00,2025-12-14 22:00:00,7.00,Horizontal Directional Drilling,Switch pilot head with reaming heads,HDD02,HDD04,"{'daylight_required': False, 'tide_window_requ..."
9,HDD04,Reaming of short stop,2025-12-14 22:00:00,2025-12-15 10:00:00,12.00,Horizontal Directional Drilling,Reaming of Bore to final diameter (short stop ...,HDD03,HDD05,"{'daylight_required': False, 'tide_window_requ..."


In [38]:
# Load the tidal data
file_path = os.path.join(main_input_folder_path,"Reference\harwich_tide.csv")
tide_df = pd.read_csv(file_path, skiprows=3, names=["DateTime", "Height"])

# Convert the 'DateTime' column to datetime format
tide_df['DateTime'] = pd.to_datetime(tide_df['DateTime'], errors='coerce')

# Filter the data to include only rows where the month is December
december_tide_df = tide_df[tide_df['DateTime'].dt.month == 12]

tide_window_df = events


<>:2: SyntaxWarning:

invalid escape sequence '\h'

<>:2: SyntaxWarning:

invalid escape sequence '\h'

C:\Users\webbb\AppData\Local\Temp\ipykernel_20928\2389328646.py:2: SyntaxWarning:

invalid escape sequence '\h'



In [39]:
fig = plot_weather(schedule_df, scheduler.daylight_windows, december_tide_df, tide_window_df)

In [40]:
outputpath=os.path.join(main_input_folder_path, "pipe_pull_schedule.html")
fig.write_html(outputpath, include_plotlyjs='cdn')